In [ ]:
# An attempt to model RNA evolution using a large language model (LLM)
# In config/rna_rag.yaml, set prompt_choice: 1 to generate RNA sequences
# go to cd drfold2 and start drfold_container
# docker run --gpus all -it --name drfold_container -v .:/opt/drfold2 drfold_image bash

# Load configuration file and check for appropriate prompt
import yaml
with open("../configs/rna_rag.yaml", 'r') as f:
    global_config = yaml.safe_load(f)
assert(global_config["llm_model"]["prompt_choice"]==1)

In [ ]:
import os
import sys
import time
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../RiboDiffusion'))

import matplotlib.pyplot as plt
from PIL import Image

from lib.fasta_utils import read_single_fasta, write_single_fasta

from importlib import reload

import lib.lib_drfold
reload(lib.lib_drfold)  # Reload the module
from lib.lib_drfold import clear_output, fasta2pdb

import lib.lib_ribodiffusion
reload(lib.lib_ribodiffusion)  # Reload the module
from lib.lib_ribodiffusion import Pdb2Fasta
pdb2fasta = Pdb2Fasta()

In [ ]:
# RAG class instantiation

from importlib import reload
from lib.RAG_Biomistral.RAG_Biomistral import RAG_Biomistral
reload(lib.RAG_Biomistral.RAG_Biomistral)  # Reload the module
from lib.RAG_Biomistral.RAG_Biomistral import RAG_Biomistral

rag = RAG_Biomistral()

# Calculate embeddings for RNA books
#path2rnaBooks =  ["../data/raw/books/mattick.pdf",] # pdfs are here
path2rnaBooks = [os.path.join(r, f) for r, _, files in os.walk("../data/raw/books") for f in files if f.lower().endswith(".pdf")]
if len(path2rnaBooks)==0:
    raise FileNotFoundError("No PDF books found. Please add PDF files to ../data/raw/books.")
path2bookEmbed = '../data/processed/RAG/book_embeddings.pkl' # Save embeddings to save time on future runs
rag.calc_book_embeddings(path2rnaBooks=path2rnaBooks, path2bookEmbed=path2bookEmbed, save=True)

In [ ]:
# Define folders, files, RAG/LLM initialization

allowed_nucleotides = {'A', 'U', 'G', 'C'}

input_fasta_dir = "../data/processed/RAG/rag_pipeline/"
output_dir = "../data/processed/RAG/RNA_evolution/"
os.makedirs(output_dir, exist_ok=True)

fasta_files = ["trna_glycine"]
num_evolution_steps = 200
for i in range(num_evolution_steps):
    fasta_files.append(str(i))

sequence_0 = read_single_fasta(input_fasta_dir+fasta_files[0]+".fasta")[1]
print("Initial sequence", sequence_0)
clear_output(output_dir)
write_single_fasta(header="seq_"+str(len(sequence_0))+"nt", sequence=sequence_0, filepath=output_dir+fasta_files[0]+".fasta")

rag.preprocess() 

In [ ]:
# Loop through evolution steps
# Output saved to: ../data/processed/RAG/RNA_evolution/

import lib.lib_x3dna
reload(lib.lib_x3dna)  # Reload the module
from lib.lib_x3dna import RnaProperties

for i in range(len(fasta_files)-1):

    print("-----> STEP", i)
    time.sleep(1)

    print("- calculate PDB from fasta")
    fasta_file = output_dir + fasta_files[i] + ".fasta"
    sequence = read_single_fasta(fasta_file)[1]
    fasta2pdb(source_path = fasta_file)
    pdb_file = fasta_file.replace('.fasta', '.pdb')

    print()
    print("- calculating PDB features...")
    fromPDB = RnaProperties(sequence=sequence, pdb_input=pdb_file)
    PDBprompt = {}
    PDBprompt["general_info"] = fromPDB.get_genInfo()
    PDBprompt["helices_info"] = fromPDB.get_helicesInfo() # "" if no helices
    # GET and save 3D structure image
    pdbImage = fromPDB.get_pdbImage()
    image_file = pdb_file.replace('.pdb', '.png')
    plt.imsave(image_file, pdbImage, cmap='viridis')
    rag_images = [Image.open(image_file).convert('RGB')]

    print()
    print("- calculating backfolded_rna")
    output = pdb2fasta.pdb2fasta(source_path = pdb_file, cond_scale=-1.)
    output_sorted = sorted(output, key=lambda x: x[1], reverse=True) # sort by similarity to main_sequence

    print()
    print("- LLM part...")
    index_backfolded_rna = 0
    while index_backfolded_rna<len(output_sorted):
        # Generate the ancestral RNA sequence
        backfolded_rna_tuple = output_sorted[index_backfolded_rna]
        print("XXXXXX sequence:", backfolded_rna_tuple[0], backfolded_rna_tuple[1])
        answer = rag.process(main_sequence=sequence, backfolded_rna_tuple=backfolded_rna_tuple, 
            PDBprompt=PDBprompt, rag_images=rag_images)
        print(sequence)
        sequence_llm = answer.split("\n")[-1].strip().split(" ")[-1].replace('"', '').replace(':', '')

        # Check whether the ancestral RNA sequence is correct
        true_sequence = all(nucleotide in allowed_nucleotides for nucleotide in sequence_llm)
        if (sequence_llm==sequence) or (not true_sequence) or (sequence_llm==""):
            index_backfolded_rna +=1
            print("Problems with RNA generation by llm. Trying again with index_backfolded_rna = {}".format(index_backfolded_rna))
        else: # (sequence_llm != sequence) and true_sequence
            print("Valid RNA sequence, generated by LLM, on attempt {}".format(index_backfolded_rna))
            sequence = sequence_llm
            break     
    print(sequence)

    fasta_file_next = output_dir + fasta_files[i+1] + ".fasta"
    write_single_fasta(header="seq", sequence=sequence, filepath=fasta_file_next)




In [ ]:
# Compare two RNA sequences

from Bio.Align import PairwiseAligner

# Example RNA sequences
rna1 = "GGGGCCAUAGCUCAAUUGGCAGAGCGCCGCCUUUGCAAGGCGGAGGCUAGGGGUUCGAUUCCCCUUGGCUCCA" 
rna2 = "GGGGCCAUAGCUCAAUUGGCAGAGCGCCGCCUUUGCAAGGUAGAGGUCCGGGGUUCGAAUCCCCGCGCCUCCC" 



# Create and Configure the Aligner
aligner = PairwiseAligner()

# Optional: adjust scoring
aligner.match_score = 2
aligner.mismatch_score = -1
aligner.open_gap_score = -2
aligner.extend_gap_score = -0.5

# Run the Alignment
alignments = aligner.align(rna1, rna2)

# Print the Best Alignment
best_alignment = alignments[0]
print(type(best_alignment))
print(best_alignment)
print(f"Score: {best_alignment.score}")